## 1 样本与标注  

### Display Samples  

Mask R-CNN 中使用的训练数据是 COCO 数据集，Mask 的标注了物体所在的位置及边缘信息，不像物体检测任务中，物体位置标注的是一个矩形框  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/1.jpg?raw=true)
<center> **Mask 标注** </center >
### Bounding Boxes  

没有使用源数据集中用于目标检测的边界框标注坐标，而是使用 Mask 来计算边界框。从而可以在不同的源数据集上，统一的处理边界框，而且，因为只是从更新后的 Mask 生成边界框，而不用根据不同的转换去调整边界框的坐标，所以还可以更轻松地调整大小，旋转或裁剪图像。  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/2.jpg?raw=true =100x)
<center> **目标边界框** </center >

### Image Resize  

为了支持每批次的多个图像，图像被调整为一个统一的尺寸（1024x1024）。保持图像的长宽比，如果图像不是正方形，则在顶部/底部或者左/右进行零填充。  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/3.jpg?raw=true =100x)
<center> **Resize 之后的图像** </center >

### Mini Masks  

使用高分辨率图像进行训练时，实例二进制 Mask 可能会变大。例如，如果使用 1024x1024 图像训练，那么单个实例的 Mask 需要 1MB 的内存。如果一个图像有 100 个实例，那么就有 100MB 的Mask。  

为了提高训练速度，我们通过以下方式优化 Mask：  
* 我们存储目标边界框内的 Mask 像素，而不是完整图像的 Mask 。大多数物体与图像大小相比明显较小，所以我们通过不在物体周围存储很多零来节省空间。  
* 我们将 Mask 调整为较小的尺寸（例如56x56）。对于大于所选大小的对象，会失去一些准确性。但是大多数对象标注并不是非常准确，所以对于大多数情况来说，这种损失是可以忽略的。  

为了可视化掩码大小调整的效果，并验证代码的正确性，可以看到一些示例，如下所示。  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/4.jpg?raw=true =100x)
<center> **Mask 调整大小** </center >

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/4.jpg?raw=true =100x)
<center> **Mask 调整大小之后映射回原图** </center >

### Anchors

anchors 的顺序很重要。在训练和预测阶段使用相同的顺序。它必须匹配卷积执行的顺序。  

对于一个 FPN 网络来说，anchors 的排列方式必须能够很容易地将 anchors 与预测 anchors 分数和移位的卷积层输出相匹配。  
* 先按金字塔等级排序。 第一级的所有 anchors，然后是第二级的所有 anchors，等等。这使得通过级别分离 anchors 更容易。  
* 每层中，按照特征图处理的顺序对 anchors 进行排序。通常，卷积层从左上角开始并向右逐行移动的处理特征图。
* 对于每个特征图单元格，为不同比例的 anchors 选择排序顺序。这里我们以传递给函数的比例顺序来对 anchors 进行排序。  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/6.jpg?raw=true)
<center> **中间像素点上的 anchors ** </center >

**Anchor Stride:**

在 FPN 架构中，前几层的特征映射是高分辨率的。例如，如果输入图像是1024x1024，那么第一层的特征图是 256x256，产生大约 200K 的 anchors（256 * 256 * 3）。这些 anchors 是 32×32 像素，它们相对于图像像素的步幅是 4 个像素，因此有很多重叠。如果我们以一定的步长为特征地图中的单元格生成 anchors ，可以显著的减少负载。例如，2 的步幅将削减四倍的 anchors 数量。  


**Show positive anchors**  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/7.jpg?raw=true)
<center> ** positive anchors ** </center >
**Show negative anchors**  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/8.jpg?raw=true)
<center> ** negative anchors ** </center >
**Show neutral anchors**  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/Mask-pics/9.jpg?raw=true)
<center> ** neutral anchors ** </center >

## 2 处理流程  

### Stage 1: Region Proposal Network  

区域提议网络（RPN）在图像上许多 anchors 上运行一个轻量级的二进制分类器，并返回对象/无对象分数。具有高对象分数的 anchors 被传递到第二阶段进行分类。  

通常，即使是正类的 anchors 也不能完全覆盖物体。因此，RPN 也会对位移（位置和大小的增量）进行回归，以便将其应用于 anchors，并将其重新调整到目标的正确边界。
  

#### 1.a RPN Targets  

RPN 目标是 RPN 的训练值。为了生成目标，我们从一个覆盖不同尺度的完整图像的网格开始，然后计算不同的 anchors 与 ground truth 之间的 IOU。 Positive anchors 是指与任意一个目标的 ground truth 之间的 IoU > 0.7的 anchors。negative anchors 是指与所有目标的 ground truth 之间的 IoU < 0.3 的 anchors。夹在两者之间（即 0.3 < IOU < 0.7）被认为是中性的，并且被排除在训练之外。  

为了训练RPN回归器，我们还计算了需要做的移动和调整，使 anchors 完全覆盖 ground truth 目标位置。  

```
target_rpn_match         shape: (65472,) #在大小为1024x1024的图像上共生成65472个anchors,按照上面的条件对这些anchor进行分类(正、负、中)

target_rpn_bbox          shape: (256, 4) #取256个anchors 训练 rpn

positive_anchors         shape: (14, 4)  #例如，下图共有14个anchor是正的，即前景

negative_anchors         shape: (242, 4) # 有采样242个anchors 作为背景

neutral anchors          shape: (65216, 4)

refined_anchors          shape: (14, 4)  # 对这些anchor 计算与 ground truth 之间的偏移，进行校准，rpn 也需要对这个偏移进行学习

```
```
Display positive anchors before refinement (dotted) and
after refinement (solid).
```


#### 1.b RPN Predictions  

运行 RPN 网络，展示运行的结果  

```
rpn_class                shape: (1, 65472, 2)         min:    0.00000  max:    1.00000
# rpn_class 是对 anchors 进行分类（前景、背景）的概率值
pre_nms_anchors          shape: (1, 10000, 4)         min: -362.03867  max: 1258.03870
# 取概率最高的前10000个anchors进行后续操作
refined_anchors          shape: (1, 10000, 4)         min: -1385.67920  max: 2212.44043
# 偏移矫正
refined_anchors_clipped  shape: (1, 10000, 4)         min:    0.00000  max: 1024.00000
# 限制大小
post_nms_anchor_ix       shape: (1000,)               min:    0.00000  max: 1477.00000
# 进行 NMS，限制1000个 anchors，不足的话进行 padding 填充
proposals                shape: (1, 1000, 4)          min:    0.00000  max:    1.00000
# rpn 送到下一步的 proposals 
```

```
# Show top anchors by score (before refinement)
limit = 100
```

```
# Show top anchors with refinment. Then with clipping to image boundaries
#limit = 50
```

```
# Show final proposals
# These are the same as the previous step (refined anchors 
# after NMS) but with coordinates normalized to [0, 1] range.
# limit = 50
```

### Stage 2: Proposal Classification  

这一阶段对 region proposals 进行分类   

#### 2.a Proposal Classification  

运行分类器网络，生成 region proposals 的类别概率，并调整他们的位置  

```
proposals                shape: (1, 1000, 4)          min:    0.00000  max:    1.00000
# 送入检测网络的proposals，共1000个，以及这些anchors的坐标
probs                    shape: (1, 1000, 81)         min:    0.00000  max:    0.99999
# proposals 属于，每一类的概率
deltas                   shape: (1, 1000, 81, 4)      min:   -3.26400  max:    3.83929
# proposals 的偏移
masks                    shape: (1, 100, 28, 28, 81)  min:    0.00000  max:    0.99984
# 计算出来的 Mask ，NMS 之后，限制100个，不足的进行padding填充
detections               shape: (1, 100, 6)           min:    0.00000  max:  844.00000
# 检测结果
```

#### 2.c Step by Step Detection  

检测的流程  

进行边界框位置的进行调整  

去掉低置信度的目标  

对每个类别进行非极大值抑制  

### Stage 3: Generating Masks  

这个阶段从上一层获取检测结果（调整后的边界框和类别 ID），并运行 Mask 分支为每个实例生成分割 Mask。  

#### 3.a Mask Targets

这是用来训练的 Mask 标注  

这是生成出来的 Mask 

#### 3.b Predicted Masks  

